In [1]:
import requests
from bs4 import BeautifulSoup
import json
import unicodedata
import re
from unidecode import unidecode
import os
import unicodedata

In [2]:
# Read the Google API key from the file
with open('google_api', 'r') as file:
    GOOGLE_API_KEY = file.read().strip()

In [3]:
def extract_movie_links():
    url = "https://www.athinorama.gr/cinema/guide/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all div elements with class "item horizontal card-item"
    movie_cards = soup.find_all('div', class_='item horizontal card-item')
    movie_links = []
    for card in movie_cards:
        # Find the link inside item-title div
        title_div = card.find('h2', class_='item-title')
        if title_div:
            link = title_div.find('a')
            if link and link.get('href'):
                movie_links.append(link['href'])
    
    return movie_links


def get_movie_times(url):
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # Find all inner-panel divs
    panels = soup.find_all("div", class_="panel-inner")

    results = []
    for panel in panels:
        # Get all elements with class daytimeschedule inside each inner-panel
        schedules = panel.find_all(class_="daytimeschedule")
        times = [s.get_text(strip=True) for s in schedules]
        if times:
            results.append(times)
    # Print extracted schedules
    for i, sched in enumerate(results, 1):
        print(f"Panel {i}: {sched}")
        

def get_movie_theater(url):

    # fetch the page
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # find all theater blocks
    theaters = []
    for title_tag, details_tag in zip(
        soup.find_all("h2", class_="item-title"),
        soup.find_all("div", class_="details")):
        name = title_tag.get_text(strip=True)
        address = details_tag.get_text(" ", strip=True)  # keep spacing
        theaters.append({"name": name, "address": address})

    # print results
    for t in theaters:
        print(f"{t['name']} - {t['address']}")



def is_greek(text):
    """Return True if text contains mostly Greek characters."""
    greek_chars = re.findall(r"[Α-Ωα-ωάέήίόύώΆΈΉΊΌΎΏ]", text)
    return len(greek_chars) > len(text) * 0.5  # >50% Greek letters = Greek text

def transliterate_greek_to_latin(text):
    """Convert Greek to Latin using unidecode if needed."""
    if is_greek(text):
        return unidecode(text)
    return text

def get_cinema_website_from_google_places(name: str, address: str = None):
    """Fetch cinema website URL from Google Places API."""
    import time
    
    # Step 1: Search for the place to get place_id
    search_query = name if not address else f"{name}, {address}"
    
    # First, try to find the place using Places Search API
    search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    search_params = {
        "query": search_query,
        "key": GOOGLE_API_KEY,
        "language": "el",
        "type": "movie_theater"  # Specify we're looking for cinemas
    }
    
    try:
        search_response = requests.get(search_url, params=search_params)
        search_response.raise_for_status()
        search_data = search_response.json()
        
        if search_data["status"] != "OK" or not search_data.get("results"):
            print(f"⚠️ No place found for '{search_query}'")
            return {"website": None}
        
        # Get the first (most relevant) result
        place = search_data["results"][0]
        place_id = place.get("place_id")
        
        if not place_id:
            print(f"⚠️ No place_id found for '{search_query}'")
            return {"website": None}
        
        # Step 2: Get detailed information including website
        details_url = "https://maps.googleapis.com/maps/api/place/details/json"
        details_params = {
            "place_id": place_id,
            "fields": "website",  # Only request website field
            "key": GOOGLE_API_KEY,
            "language": "el"
        }
        
        # Add a small delay to respect API rate limits
        time.sleep(0.1)
        
        details_response = requests.get(details_url, params=details_params)
        details_response.raise_for_status()
        details_data = details_response.json()
        
        if details_data["status"] != "OK":
            print(f"⚠️ Could not get details for '{search_query}' (place_id: {place_id})")
            return {"website": None}
        
        result = details_data.get("result", {})
        website = result.get("website")
        
        print(f"✅ Found info for '{name}': {website or 'No website'}")
        
        return {"website": website}
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error for '{search_query}': {e}")
        return {"website": None}
    except Exception as e:
        print(f"❌ Unexpected error for '{search_query}': {e}")
        return {"website": None}


def get_cinema_info_from_google(name: str, address: str = None):
    """Fetch cinema info (lat, lon, area, formatted address) from Google Maps API."""
    query = name if not address else f"{name}, {address}"
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": query,
        "key": GOOGLE_API_KEY,
        "language": "el"  # or "en" depending on what you want
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data["status"] != "OK" or not data["results"]:
        print(f"⚠️ Google Maps API: No match for '{query}'")
        return {
            "lat": None,
            "lon": None,
            "area": "Unknown",
            "formatted_address": None
        }

    result = data["results"][0]
    geometry = result["geometry"]["location"]
    address_components = result.get("address_components", [])

    # Try to extract area (e.g., neighborhood, locality, sublocality)
    area = "Unknown"
    # Extract broader area (default: locality)
    area = next(
        (c["long_name"] for c in address_components if "locality" in c["types"]),
        "Unknown"
    )

    formatted_addr = result.get("formatted_address")
    
    # 🧹 Step 3: Remove Greek street words and abbreviations
    first_part = re.sub(
        r'\b(Λ\.?|Λεωφόρος|Λεωφ\.?|Οδός|Οδ\.?|Δρόμος|Δρ\.?)\b',
        '',
        formatted_addr,
        flags=re.IGNORECASE
    ).strip()

    # 🧹 Step 4: Keep only up to the first '&' or 'και' or '-' (e.g., "Συγγρού & Φραντζή" → "Συγγρού")
    first_part = re.split(r'\s*&\s*|\s*και\s*|\s*-\s*', first_part)[0].strip()
    
    # --- Geocoding ---
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": f"{first_part}",
        "format": "json",
        "addressdetails": 1,
        "limit": 1
    }

    
    r = requests.get(url, params=params, headers={"User-Agent": "cinema-app"})
    r.raise_for_status()
    data = r.json()
    if data:
        details = data[0].get("address", {})
        open_info_suburb = details.get("suburb")
        open_info_neighbourhood = details.get("neighbourhood")
    else:
        open_info_suburb = open_info_neighbourhood = ""
        
    

    return {
        "lat": geometry["lat"],
        "lon": geometry["lng"],
        "area": area,
        "suburb": open_info_suburb,
        "neighbourhood": open_info_neighbourhood,
        "formatted_address": formatted_addr
        }


def geocode_area(address):
    # 🧹 Step 1: Remove parentheses and contents inside them
    cleaned = re.sub(r'\([^)]*\)', '', address).strip()

    # 🧹 Step 2: Keep only the first comma-separated part
    # first_part = cleaned.split(',')[0].strip()

    # 🧹 Step 3: Remove Greek street words and abbreviations
    first_part = re.sub(
        r'\b(Λ\.?|Λεωφόρος|Λεωφ\.?|Οδός|Οδ\.?|Δρόμος|Δρ\.?)\b',
        '',
        first_part,
        flags=re.IGNORECASE
    ).strip()

    # 🧹 Step 4: Keep only up to the first '&' or 'και' or '-' (e.g., "Συγγρού & Φραντζή" → "Συγγρού")
    first_part = re.split(r'\s*&\s*|\s*και\s*|\s*-\s*', first_part)[0].strip()

#     # 🧹 Step 5: Keep only first word and possible number (e.g. "Παπανδρέου 12")
#     match = re.match(r'^([\wΆ-ώΑ-Ωά-ώ]+(?:\s*\d{1,3})?)', first_part)
#     if match:
#         query_base = match.group(1)
#     else:
#         query_base = first_part

    # 🧹 Step 6: Collapse spaces
    query_base = re.sub(r'\s+', ' ', query_base).strip()

    # --- Geocoding ---
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": f"{query_base}",
        "format": "json",
        "addressdetails": 1,
        "limit": 1
    }

#     print(f"Geocoding query: {params['q']}")

    try:
        r = requests.get(url, params=params, headers={"User-Agent": "cinema-app"})
        r.raise_for_status()
        data = r.json()
        if not data:
            return None

        details = data[0].get("address", {})
        area = (
            details.get("suburb")
            or details.get("neighbourhood")
            or details.get("city_district")
            or details.get("town")
            or details.get("city")
        )

        lat = data[0].get("lat")
        lon = data[0].get("lon")

        return {"area": area, "lat": lat, "lng": lon}

    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None


def normalize_name(name: str) -> str:
    """Normalize and clean cinema name for reliable matching."""
    if not name:
        return ""
    # Normalize Unicode (remove accent inconsistencies)
    name = unicodedata.normalize("NFKC", name)
    # Remove invisible spaces, trim, and lowercase
    name = name.strip().replace("\u200b", "").replace("\xa0", " ").lower()
    return name

def load_cinema_database(filename="cinema_database.json"):
    """Load existing cinema database from file."""
    if os.path.exists(filename):
        try:
            with open(filename, "r", encoding="utf-8") as f:
                return json.load(f)
        except (json.JSONDecodeError, FileNotFoundError):
            print(f"⚠️ Warning: {filename} is empty or corrupted. Starting fresh.")
            return {}
    else:
        print(f"ℹ️ No existing {filename} found. Starting fresh.")
        return {}

def save_cinema_database(cinema_db, filename="cinema_database.json"):
    """Save cinema database to file."""
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(cinema_db, f, ensure_ascii=False, indent=2)
    print(f"✅ Cinema database saved to {filename}")

def get_or_create_cinema_info(name, address, cinema_db):
    """
    Get cinema info from database or fetch from Google API if not exists.
    Now includes website information from Google Places API.
    Returns cinema info dict and updates the database.
    """
    # Create a unique key for the cinema
    norm_name = normalize_name(name)
    norm_address = normalize_name(address) if address else ""
    cinema_key = f"{norm_name}_{norm_address}"
    
    # Check if cinema already exists in database
    if cinema_key in cinema_db:
        existing_info = cinema_db[cinema_key]
        
        # Check if we already have complete info (including website)
        if "website" in existing_info:
            print(f"✅ Found cached info (with website) for: {name}")
            return existing_info
        else:
            print(f"🔄 Found cached location info for: {name}, fetching website...")
            # Get website info and merge with existing
            website_info = get_cinema_website_from_google_places(name, address)
            merged_info = {**existing_info, **website_info}
            cinema_db[cinema_key] = merged_info
            return merged_info
    
    # Cinema not found, fetch both location and website info
    print(f"🔍 Fetching new info (location + website) for: {name}")
    
    # Get location info from Google Maps Geocoding API
    location_dict = get_cinema_info_from_google(name, address)
    
    # Get website info from Google Places API
    website_dict = get_cinema_website_from_google_places(name, address)
    
    # Merge the information
    if location_dict:
        region_dict = {**location_dict, **website_dict}
    else:
        # Fallback if location info fails
        region_dict = {
            "lat": 0,
            "lon": 0,
            "area": "Unknown",
            "suburb": "Unknown",
            "neighbourhood": "Unknown",
            "formatted_address": address,
            **website_dict
        }
    
    # Store in database (if value)
    if region_dict:
        cinema_db[cinema_key] = region_dict
    
    return region_dict



def get_movie_theater_times(url, cinema_db):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    movies_data = []
    cinemas_data = []
    
    # --- Movie Titles ---
    title_greek_tag = soup.find("h1")
    title_greek = title_greek_tag.get_text(strip=True) if title_greek_tag else "Unknown Title"

            
    card = soup.find_all("ul", class_="review-details")
    for c in card:
        original_tag = c.find("span", class_="original-title")
        if original_tag:
            original_title = original_tag.get_text()
        else:
            original_title = ""
            
    imdb = soup.find('a',class_="imdb")
    imdb = imdb.get('href') if imdb else None
        

    movies_data.append({
        "greek_title": title_greek,
        "original_title": original_title,
        "athinorama_link": url,
        "imdb_link": imdb
    })

    # --- Cinema Entries ---
    cinema_blocks = soup.find_all("div", class_="item card-item")
    for block in cinema_blocks:
        name_tag = block.find("h2", class_="item-title")
        details_tag = block.find("div", class_="details")
        name = name_tag.get_text(strip=True) if name_tag else None

        # Rooms
        rooms = []
        for panel in block.find_all("div", class_="grid schedule-grid"):
            room_name_tag = panel.find("span")
            room_name = room_name_tag.get_text(strip=True) if room_name_tag else "Main Room"
            rooms.append({"room": room_name})

        # Timetable
        room_timetable = []
        innerpanels = block.find_all("div", class_="panel-inner")
        for panel in innerpanels:
            schedules = panel.find_all(class_="daytimeschedule")
            times = [s.get_text(strip=True) for s in schedules]
            if times:
                room_timetable.append(times)
                
        address = details_tag.get_text(" ", strip=True) if details_tag else None
        # --- Get cinema info from cache or API ---
        region_dict = get_or_create_cinema_info(name, address, cinema_db)
        
        # Get values with safe .get() method, leveraging the dict guarantee
        final_area = region_dict.get('area', 'Unknown')
        suburb = region_dict.get('suburb','Unknown')
        neighbourhood = region_dict.get('neighbourhood','Unknown')

        # 1. When area is "Αθηνα", list subarea if available, otherwise use "Αθηνα (Κεντρο)"
        if final_area == "Αθήνα":
            print(region_dict)
            # Check if suburb is not empty/None AND not the same as the main area
            if suburb and normalize_name(suburb) != normalize_name(final_area):
                final_area = suburb
            elif not suburb and normalize_name(neighbourhood) != normalize_name(final_area):
                final_area = neighbourhood
            else:
                # This will act as the filter for all Athens cinemas
                final_area = "Αθήνα (Κέντρο)"

        # 2. Replace 'ampelokipi' with 'Αμπελοκηποι'
        if final_area == "Ampelokipi":
            final_area = "Αμπελόκηποι"

        # --- END OF MODIFICATION ---

        cinemas_data.append({
                "cinema": name,
                "address": region_dict['formatted_address'],
                "lat" :region_dict['lat'],
                "lon" :region_dict['lon'],
                "region": final_area,
                "subregion": region_dict['suburb'],
                "neighbourhood": region_dict['neighbourhood'],
                "website": region_dict['website'],
                "rooms": rooms,
                "timetable": room_timetable
            })

    return movies_data, cinemas_data

In [4]:
movie_links =[]

base_url = "https://www.athinorama.gr"
if __name__ == "__main__":
    links = extract_movie_links()
    for link in links:
        print(link)
        movie_links.append(base_url + link)  

/cinema/movie/monsta_x_connect_x_in_cinemas-10088246
/cinema/movie/seventeen_world_tour_new_in_Japan_live_viewing-10088245
/cinema/movie/i_agapi_pou_apomenei-10088113
/cinema/movie/anoiksi_kalokairi_fthinoporo_xeimonas_kai_anoiksi-1007265
/cinema/movie/axinos-10087613
/cinema/movie/bermilio_i_nufi_tou_bounou-10086358
/cinema/movie/bougonia-10087837
/cinema/movie/geia_sou_frinta-10088097
/cinema/movie/i_geusi_tou_kerasiou-1005620
/cinema/movie/gia_panta-10088223
/cinema/movie/ghost_in_the_shell-10088243
/cinema/movie/ghost_in_the_shell_2_innocence-10088244
/cinema/movie/grand_prix_o_guros_tis_europis-10087846
/cinema/movie/wicked_meros_deutero-10088098
/cinema/movie/to_dakru_tou_diabolou-10088224
/cinema/movie/duo_kapnismenes_kannes-1006038
/cinema/movie/ena_aplo_atuxima-10087413
/cinema/movie/eksolothreutis_aggelos-1007248
/cinema/movie/zooupoli_2-10088225
/cinema/movie/kaiti_drosou_kibotio_mnimis-10087669
/cinema/movie/ta_kaka_paidia_2-10086927
/cinema/movie/kamia_alli_epilogi-1008800

In [5]:
# Load cinema database at the start
cinema_database = load_cinema_database()

movies_l = []
cinemas_l = []


for url in movie_links:
    print(url)
    movie, cinema_t = get_movie_theater_times(url, cinema_database)
    movies_l.append(movie)
    cinemas_l.append(cinema_t)

https://www.athinorama.gr/cinema/movie/monsta_x_connect_x_in_cinemas-10088246
✅ Found cached info (with website) for: Village 15 Cinemas @ The Mall
https://www.athinorama.gr/cinema/movie/seventeen_world_tour_new_in_Japan_live_viewing-10088245
✅ Found cached info (with website) for: Village 15 Cinemas @ The Mall
https://www.athinorama.gr/cinema/movie/i_agapi_pou_apomenei-10088113
✅ Found cached info (with website) for: Newman Cinema
{'lat': 37.9929141, 'lon': 23.7682109, 'area': 'Αθήνα', 'suburb': 'Αμπελόκηποι', 'neighbourhood': 'Ερυθρός Σταυρός', 'formatted_address': 'Σεβαστουπόλεως 117, Αθήνα 115 26, Ελλάδα', 'website': 'https://www.newmancinema.gr/'}
https://www.athinorama.gr/cinema/movie/anoiksi_kalokairi_fthinoporo_xeimonas_kai_anoiksi-1007265
✅ Found cached info (with website) for: Ατενέ
{'lat': 38.0209212, 'lon': 23.7296286, 'area': 'Αθήνα', 'suburb': 'Πατήσια', 'neighbourhood': None, 'formatted_address': 'Άνω Πατήσια, Αθήνα, Ελλάδα', 'website': None}
https://www.athinorama.gr/ci

✅ Found cached info (with website) for: Village 15 Cinemas @ The Mall
✅ Found cached info (with website) for: Νανά Cinemax
✅ Found cached info (with website) for: @Escape Center
✅ Found cached info (with website) for: Village Shopping And More
https://www.athinorama.gr/cinema/movie/duo_kapnismenes_kannes-1006038
✅ Found cached info (with website) for: Cinobo Πατησίων
{'lat': 37.9926579, 'lon': 23.7313348, 'area': 'Αθήνα', 'suburb': 'Κυψέλη', 'neighbourhood': 'Άγιος Παντελεήμονας', 'formatted_address': '28ης Οκτωβρίου 79, Αθήνα 104 34, Ελλάδα', 'website': 'https://www.cinobopatision.gr/'}
https://www.athinorama.gr/cinema/movie/ena_aplo_atuxima-10087413
✅ Found cached info (with website) for: Άστορ
{'lat': 37.97956550000001, 'lon': 23.7327058, 'area': 'Αθήνα', 'suburb': None, 'neighbourhood': 'Ακαδημία', 'formatted_address': 'Σταδίου 28, Αθήνα 105 64, Ελλάδα', 'website': 'http://www.astorcinema.gr/'}
✅ Found cached info (with website) for: Newman Cinema
{'lat': 37.9929141, 'lon': 23.7682

✅ Found cached info (with website) for: Έλλη
{'lat': 37.9826891, 'lon': 23.733503, 'area': 'Αθήνα', 'suburb': None, 'neighbourhood': 'Ακαδημία', 'formatted_address': 'Ακαδημίας 64, Αθήνα 106 79, Ελλάδα', 'website': 'https://www.facebook.com/ellicinema/'}
✅ Found cached info (with website) for: Γαλαξίας Cinema
{'lat': 37.9850165, 'lon': 23.7611995, 'area': 'Αθήνα', 'suburb': 'Αμπελόκηποι', 'neighbourhood': 'Κουντουριώτικα', 'formatted_address': 'Λεωφ. Μεσογείων 6, Αθήνα 115 27, Ελλάδα', 'website': 'https://galaxiascinemas.gr/'}
✅ Found cached info (with website) for: Αβάνα
✅ Found cached info (with website) for: Διάνα
✅ Found cached info (with website) for: Κηφισιά Cinemax 3
✅ Found cached info (with website) for: Village 15 Cinemas @ The Mall
✅ Found cached info (with website) for: Τρία Αστέρια 3D Digital
✅ Found cached info (with website) for: Αελλώ Cinemax
{'lat': 37.9993861, 'lon': 23.7333922, 'area': 'Αθήνα', 'suburb': 'Κυψέλη', 'neighbourhood': 'Πλατεία Αμερικής', 'formatted_addre

✅ Found cached info (with website) for: Σινέ Αλίκη (Δημ. Κιν/φος)
https://www.athinorama.gr/cinema/movie/i_summoria_ton_magon_3-10088101
✅ Found cached info (with website) for: Village 15 Cinemas @ The Mall
✅ Found cached info (with website) for: Village Cinemas Παγκράτι
✅ Found cached info (with website) for: Village Cinemas Athens Metro Mall - Ag. Dimitrios
✅ Found cached info (with website) for: WestCity Cinemas
✅ Found cached info (with website) for: @Escape Center
✅ Found cached info (with website) for: Village Shopping And More
https://www.athinorama.gr/cinema/movie/sunaisthimatiki_aksia-10087020
✅ Found cached info (with website) for: Άστορ
{'lat': 37.97956550000001, 'lon': 23.7327058, 'area': 'Αθήνα', 'suburb': None, 'neighbourhood': 'Ακαδημία', 'formatted_address': 'Σταδίου 28, Αθήνα 105 64, Ελλάδα', 'website': 'http://www.astorcinema.gr/'}
✅ Found cached info (with website) for: Διάνα
✅ Found cached info (with website) for: Δημ. Κιν. Σινεάκ
https://www.athinorama.gr/cinema/mo

In [6]:
# Save updated cinema database
save_cinema_database(cinema_database)

with open("cinemas.json", "w", encoding="utf-8") as f:
    json.dump(cinemas_l, f, ensure_ascii=False, indent=2)

with open("movies.json", "w", encoding="utf-8") as f:
    json.dump(movies_l, f, ensure_ascii=False, indent=2)

✅ Cinema database saved to cinema_database.json


In [7]:
### PROXEIRO ####

In [8]:
# import requests


# def get_cinema_info_from_google(name: str, address: str = None):
#     """Fetch cinema info (lat, lon, area, formatted address) from Google Maps API."""
#     query = name if not address else f"{name}, {address}"

#     url = "https://maps.googleapis.com/maps/api/geocode/json"
#     params = {
#         "address": query,
#         "key": GOOGLE_API_KEY,
#         "language": "el"  # or "en" depending on what you want
#     }

#     response = requests.get(url, params=params)
#     data = response.json()

#     if data["status"] != "OK" or not data["results"]:
#         print(f"⚠️ Google Maps API: No match for '{query}'")
#         return {
#             "lat": None,
#             "lon": None,
#             "area": "Unknown",
#             "formatted_address": None
#         }

#     result = data["results"][0]
#     geometry = result["geometry"]["location"]
#     address_components = result.get("address_components", [])

#     # Try to extract area (e.g., neighborhood, locality, sublocality)
#     area = "Unknown"
#     # Extract broader area (default: locality)
#     area = next(
#         (c["long_name"] for c in address_components if "locality" in c["types"]),
#         "Unknown"
#     )

#     open_info = geocode_area(result.get("formatted_address"))
    
#     return {
#         "lat": geometry["lat"],
#         "lon": geometry["lng"],
#         "area": area,
#         "subarea": open_info['area'],
#         "formatted_address": result.get("formatted_address")
#     }

In [9]:
# get_cinema_info_from_google('Αβάνα',"Λυκούργου 3, 234, Χαλάνδρι 152 31, Ελλάδα")

In [10]:
# geocode_area('Λυκούργου 3, Χαλάνδρι 152 31, Ελλάδα')

In [11]:
#     # --- Geocoding ---
#     url = "https://nominatim.openstreetmap.org/search"
#     params = {
#         "q": f"{'Λυκούργου 3, Χαλάνδρι 152 31, Ελλάδα'}",
#         "format": "json",
#         "addressdetails": 1,
#         "limit": 1
#     }

# #     print(f"Geocoding query: {params['q']}")

    
#     r = requests.get(url, params=params, headers={"User-Agent": "cinema-app"})
#     r.raise_for_status()
#     data = r.json()

In [12]:
# def geocode_area(address):
#     # 🧹 Step 1: Remove parentheses and contents inside them
#     cleaned = re.sub(r'\([^)]*\)', '', address).strip()

#     # 🧹 Step 2: Keep only the first comma-separated part
#     first_part = cleaned.split(',')[0].strip()

#     # 🧹 Step 3: Remove Greek street words and abbreviations
#     first_part = re.sub(
#         r'\b(Λεωφόρος|Λεωφ\.?|Οδός|Οδ\.?|Δρόμος|Δρ\.?)\b',
#         '',
#         first_part,
#         flags=re.IGNORECASE
#     ).strip()

#     # 🧹 Step 4: Keep only up to the first '&' or 'και' or '-' (e.g., "Συγγρού & Φραντζή" → "Συγγρού")
#     first_part = re.split(r'\s*&\s*|\s*και\s*|\s*-\s*', first_part)[0].strip()

#     # 🧹 Step 5: Keep only first word and possible number (e.g. "Παπανδρέου 12")
#     match = re.match(r'^([\wΆ-ώΑ-Ωά-ώ]+(?:\s*\d{1,3})?)', first_part)
#     if match:
#         query_base = match.group(1)
#     else:
#         query_base = first_part

#     # 🧹 Step 6: Collapse spaces
#     query_base = re.sub(r'\s+', ' ', query_base).strip()

#     # --- Geocoding ---
#     url = "https://nominatim.openstreetmap.org/search"
#     params = {
#         "q": f"{query_base}, Athens, Greece",
#         "format": "json",
#         "addressdetails": 1,
#         "limit": 1
#     }

#     print(f"Geocoding query: {params['q']}")

#     try:
#         r = requests.get(url, params=params, headers={"User-Agent": "cinema-app"})
#         r.raise_for_status()
#         data = r.json()
#         if not data:
#             return None

#         details = data[0].get("address", {})
#         area = (
#             details.get("suburb")
#             or details.get("neighbourhood")
#             or details.get("city_district")
#             or details.get("town")
#             or details.get("city")
#         )

#         lat = data[0].get("lat")
#         lon = data[0].get("lon")

#         return {"area": area, "lat": lat, "lng": lon}

#     except Exception as e:
#         print(f"Error geocoding {address}: {e}")
#         return None


In [13]:
# for url in movie_links:
#     print(url)
#     get_movie_theater(url)

In [14]:
# for url in movie_links:
#     print(url)
#     get_movie_theater_times(url)

In [15]:
# url = "https://www.athinorama.gr/cinema/movie/avatar_the_way_of_water-10076581"

# # fetch the page
# response = requests.get(url)
# response.raise_for_status()

# soup = BeautifulSoup(response.text, "html.parser")

# # find all theater blocks
# theaters = []
# for title_tag, details_tag in zip(
#     soup.find_all("h2", class_="item-title"),
#     soup.find_all("div", class_="details")
# ):
#     name = title_tag.get_text(strip=True)
#     address = details_tag.get_text(" ", strip=True)  # keep spacing
#     theaters.append({"name": name, "address": address})

# # print results
# for t in theaters:
#     print(f"{t['name']} - {t['address']}")


In [16]:
# import requests
# from bs4 import BeautifulSoup

# response = requests.get(url)
# response.raise_for_status()
# soup = BeautifulSoup(response.text, "html.parser")

# cinemas_data = []

# # loop through each cinema entry
# cinema_blocks = soup.find_all("div", class_="item card-item")
# for block in cinema_blocks:
#     # get cinema name and address
#     name_tag = block.find("h2", class_="item-title")
#     details_tag = block.find("div", class_="details")
#     name = name_tag.get_text(strip=True) if name_tag else None
#     address = details_tag.get_text(" ", strip=True) if details_tag else None

#     rooms = []
#     # find panel-inner (each one usually corresponds to a room / screen)
#     for panel in block.find_all("div", class_="grid schedule-grid"):
#         # room name (if available, otherwise fallback)
#         room_name_tag = panel.find("span")
#         room_name = room_name_tag.get_text(strip=True) if room_name_tag else "Main Room"

#         rooms.append({
#             "room": room_name
#         })
        
#     # Find all inner-panel divs
#     room_timetable = []
#     innerpanels = block.find_all("div", class_="panel-inner")
#     for panel in innerpanels:
#             # Get all elements with class daytimeschedule inside each inner-panel
#             schedules = panel.find_all(class_="daytimeschedule")
#             times = [s.get_text(strip=True) for s in schedules]
#             if times:
#                 room_timetable.append(times)        

#     cinemas_data.append({
#         "cinema": name,
#         "address": address,
#         "rooms": rooms,
#         "timetable": room_timetable
#     })

# # show results
# import json
# print(json.dumps(cinemas_data, indent=2, ensure_ascii=False))
